Using Google Sheets to collaborate is a great way to get work accomplished if you and your collaborator/s: 
1. have different schedules and upload data at different times.
2. collect different data that needs to be aggregated in the same place,
3. want to work off the same kind of data.
4. don't use a formal database for data collection.

As an active data scientist, while I would prefer to have data capture in a queryable database, sometimes that might not be an option; especially if that data is a one-off experiment. Luckily, Google Sheets has an api to which I can connect and programmatically manipulate data. This blog post will walk through some of these actions using Python.

Before jumping into the code, I am presuming you have a familiarity with Google's service account system. To quote from Google's cloud documentation a _"A service account is a special type of Google account intended to represent a non-human user that needs to authenticate and be authorized to access data in Google APIs."_ If you don't have a service account you won't be able to access 

In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import json
import boto3
import os
from faker import Faker

In [4]:
sm = boto3.client("secretsmanager")
secret_json = sm.get_secret_value(SecretId="trimble-service-account-google-apis")[
    "SecretString"
]

secret_data = json.loads(secret_json)
scopes = ["https://www.googleapis.com/auth/spreadsheets"]

creds = service_account.Credentials.from_service_account_info(
    secret_data, scopes=scopes
)
service = build("sheets", "v4", credentials=creds)
sheets = service.spreadsheets()
sheet_id = os.environ['SHEET_ID']

In [5]:
fake = Faker()
profileData = [fake.profile() for i in range(100)]
df = pd.DataFrame(profileData)
df.head()

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Automotive engineer,Davidson-Wilson,140-09-1384,"39038 Reed Well\nYoungbury, MT 35036","(-41.8482505, -39.535320)",B+,"[http://reid-ward.biz/, https://andrews.com/, ...",joseph91,Steven Harrington,M,"51694 Smith Estates Apt. 652\nMitchellhaven, S...",hthompson@hotmail.com,2004-01-12
1,Operations geologist,"Miller, Richmond and Middleton",542-08-1072,"625 Carlson Squares Apt. 750\nShirleychester, ...","(-72.871571, -67.821944)",AB+,[https://www.sanchez.com/],david48,Timothy Walter,M,"808 Michael Radial\nAlexside, NY 30322",mary33@hotmail.com,1918-09-25
2,Hospital doctor,Gonzales Ltd,603-93-1565,"78452 Angela Pike\nKimberlystad, IN 75210","(-69.363168, 48.851713)",AB-,"[https://www.sutton.info/, https://www.russo.n...",james03,William Burch,M,"9711 Foster Station\nKellybury, NE 52104",lschultz@gmail.com,1939-07-12
3,"Scientist, research (maths)",Jones Group,659-78-4784,Unit 1691 Box 4650\nDPO AP 98487,"(-67.3018705, 2.694810)",AB+,"[https://cooper.com/, https://tran.com/, http:...",katherinecarr,Darrell Marshall,M,"0581 Alexander Estate\nSouth Crystal, DE 63378",lmcconnell@hotmail.com,1932-05-31
4,"Surveyor, quantity",Williams PLC,835-78-7143,"0253 Wilkinson Stravenue\nNew Michaelside, OH ...","(-7.3012915, 148.242220)",O-,"[http://www.tate-schmitt.com/, http://www.hawk...",harrisryan,Barbara Cline,F,"43433 Lowe Ridges Apt. 950\nChangview, NV 24931",josephjohnson@gmail.com,1996-07-28


In [12]:
upload_df = df[['job', 'company', 'blood_group', 'username']]

In [20]:
upload_df.head()

,job,company,blood_group,username
0,Automotive engineer,Davidson-Wilson,B+,joseph91
1,Operations geologist,"Miller, Richmond and Middleton",AB+,david48
2,Hospital doctor,Gonzales Ltd,AB-,james03
3,"Scientist, research (maths)",Jones Group,AB+,katherinecarr
4,"Surveyor, quantity",Williams PLC,O-,harrisryan


In [13]:
values = [upload_df.columns.values.tolist()]
values.extend(upload_df.values.tolist())
data = [
    {'range' : 'Sheet1', 
     'values' : values}
]
batch_update = {
    'value_input_option': 'RAW',
    'data': data 
}

request = (
    service.spreadsheets()
    .values()
    .batchUpdate(
        spreadsheetId=sheet_id,
        body=batch_update
    )
    .execute()
)

In [15]:
_r = "Sheet1!A1:E101"

In [16]:
new_data = (
    pd.DataFrame(
        sheets.values()
        .get(spreadsheetId=sheet_id, range=_r, majorDimension='ROWS')
        .execute()['values'])
)
new_data.head()

,0,1,2,3,4
0,job,company,blood_group,username,id_number
1,Automotive engineer,Davidson-Wilson,B+,joseph91,1
2,Operations geologist,"Miller, Richmond and Middleton",AB+,david48,2
3,Hospital doctor,Gonzales Ltd,AB-,james03,3
4,"Scientist, research (maths)",Jones Group,AB+,katherinecarr,4


In [17]:
new_data.columns = new_data.iloc[0]

In [18]:
new_data.drop(index=[0], inplace=True)

In [19]:
new_data.head()

,job,company,blood_group,username,id_number
1,Automotive engineer,Davidson-Wilson,B+,joseph91,1
2,Operations geologist,"Miller, Richmond and Middleton",AB+,david48,2
3,Hospital doctor,Gonzales Ltd,AB-,james03,3
4,"Scientist, research (maths)",Jones Group,AB+,katherinecarr,4
5,"Surveyor, quantity",Williams PLC,O-,harrisryan,5
